In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random

# Parameters

In [ ]:
image_size = (128, 128)
num_classes = 5
epochs = 30
batch_size = 32

In [ ]:
data_dir = './flowers/'

class_names = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    image = image / 255.0  
    return image

images = []
labels = []

for i, class_name in enumerate(class_names):
    class_dir = os.path.join(data_dir, class_name)
    print("Class ",i,'',class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = preprocess_image(image_path)
        images.append(image)
        labels.append(i)

# Convert images and labels to numpy arrays

images = np.array(images)
labels = np.array(labels)


# Split the data into training and test sets

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.2, stratify=labels, random_state=42
)


print("Training data size:", len(train_images))
print("Test data size:", len(test_images))

#  model

In [ ]:

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])




# Train the model

In [ ]:
# model.fit(train_images, train_labels,
#           epochs=epochs,
#           batch_size=batch_size)

history = model.fit(train_images, train_labels,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(test_images, test_labels))

# Plot the Loss and Accuracy curves

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Curves')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Evaluate the model

In [ ]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

# classification report

In [ ]:
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)

classification_rep = classification_report(test_labels, predicted_labels, target_names=class_names, output_dict=True)

# Convert classification report to pandas DataFrame

df_classification = pd.DataFrame(classification_rep).transpose()

df_classification

# Confusion matrix

In [ ]:
cm = confusion_matrix(test_labels, predicted_labels)
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)

# Plot confusion matrix

plt.figure(figsize=(8, 6))
sns.heatmap(df_cm, annot=True, cmap='Blues', fmt='g')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Test the model on an input image

In [ ]:
input_image_path = './flowers/sunflower/1008566138_6927679c8a.jpg'
input_image = preprocess_image(input_image_path)
input_image = np.expand_dims(input_image, axis=0)  

predictions = model.predict(input_image)
predicted_label = np.argmax(predictions[0])
predicted_class = class_names[predicted_label]

print('Predicted class:', predicted_class)

# save model

In [ ]:
model.save('model.h5')